In [61]:
import tensorflow as tf
import numpy as np
import random
from tensorflow.examples.tutorials.mnist import input_data

#1.导入数据
mnist = input_data.read_data_sets(r'C:\Users\TH\Desktop\DL\sample of writing numbers',one_hot=True) 
        #mnist 数字识别机 1.文件夹名 2.onehot 布尔型 如果一个数组被定义为onehot 数组中有且只有一个元素为1，其他0
        #电脑要联网才能不报错

#2.属性设置
trainNum = 55000
testNum = 10000
trainSize = 5000
testSize  = 5

#3.分解数据,得到训练图片，测试图片的数据和标签
trainIndex = np.random.choice(trainNum,trainSize,replace=False)#随机选择下标,生成size个随机数，范围在0-num之间，不可重复
testIndex = np.random.choice(testNum,testSize,replace=False)
trainData = mnist.train.images[trainIndex]    #获取训练图片，500行*784列，图片个数500，图片大小28*28=784
trainLabel = mnist.train.labels[trainIndex]    #获取训练标签，500行*10列，图片个数500，标签共10个，若print，每次运行结果不一样
testData = mnist.test.images[testIndex]    #获取测试图片，5行*784列，图片个数5，，图片大小28*28=784
testLabel = mnist.test.labels[testIndex]    #获取测试图片，5行*10列，图片个数5，，标签共10个,若print，每次运行结果不一样
k = 4     #找到k个最接近的图片
print('trainData.shape=',trainData.shape)
print('trainLabel.shape=',trainLabel.shape)
print('testData.shape=',testData.shape)
print('testLabel.shape=',testLabel.shape)
print(testLabel)

#4.定义tensorflow输入数据
trainDataInput = tf.placeholder(shape=[None,784],dtype=tf.float32) #n行784列 n:图片个数 784：完整图片
trainLabelInput = tf.placeholder(shape=[None,10],dtype=tf.float32)
testDataInput = tf.placeholder(shape=[None,784],dtype=tf.float32)
testLabelInput = tf.placeholder(shape=[None,10],dtype=tf.float32)

#5.KNN距离具体计算
f1 = tf.expand_dims(testDataInput,1)#增加维度,变成5*500(默认1)*784,5张测试，500张样本，每个像素的差，为三维数据
f2 = tf.subtract(trainDataInput,f1) #描述测试图片和训练图片每个像素的差，结果为三维，放入f2中
f3 = tf.reduce_sum(tf.abs(f2),reduction_indices=2)#数据累加函数，为5*500，1：数据，计算距离有负，换成绝对值，2：在第三维进行累加784个差值

#6.从500张中找到k张最接近的图片,并且看对应哪一个label最多
f4 = tf.negative(f3)#对f3取反
f5,f6 = tf.nn.top_k(f4,k=4)#选取f4中元素最大的四个值，即f3中最小的四个值,f5f6分别为值的内容和对应图片的下标

#7.通过对应图片的下标找到对应图片的label
f7 = tf.gather(trainLabelInput,f6)

#8.统计出现最多的数字
f8 = tf.reduce_sum(f7,reduction_indices=1)#将label累加
f9 = tf.argmax(f8,dimension=1)#找到最大的label的下标

#9.检测准确率:正确数据在testLabel中，与f9进行比较,统计正确率



#运行tensorflow
with tf.Session() as sess:
    p1 = sess.run(f1,feed_dict={testDataInput:testData[0:5]})#运行f1并给参数testData
    print('p1.shape=',p1.shape)#p1=5*1*784
    
    p2 = sess.run(f2,feed_dict={trainDataInput:trainData,testDataInput:testData[0:5]})
    print('p2.shape=',p2.shape)#p2=5*500*784
    
    p3 = sess.run(f3,feed_dict={trainDataInput:trainData,testDataInput:testData[0:5]})#最核心：距离计算
    print('p3.shape=',p3.shape)#p3=5*500
    
    p4 = sess.run(f4,feed_dict={trainDataInput:trainData,testDataInput:testData[0:5]})
    print('p4.shape=',p4.shape)#p4=5*500
    
    p5,p6 = sess.run((f5,f6),feed_dict={trainDataInput:trainData,testDataInput:testData[0:5]})#同时运行f5f6
    print('p5.shape=',p5.shape)#p4=5*4
    print('p6.shape=',p6.shape)#p4=5*4
    print(p5)#测试图片分别对应500张样本最接近的值
    print(p6)#该值对应的下标
    
    p7 = sess.run(f7,feed_dict={trainDataInput:trainData,testDataInput:testData[0:5],trainLabelInput:trainLabel})
    print('p7.shape=',p7.shape)#p7=5*4*10
    print(p7)
    
    p8 = sess.run(f8,feed_dict={trainDataInput:trainData,testDataInput:testData[0:5],trainLabelInput:trainLabel})
    print('p8.shape=',p8.shape)#p8=5*10
    print(p8)
    
    p9 = sess.run(f9,feed_dict={trainDataInput:trainData,testDataInput:testData[0:5],trainLabelInput:trainLabel})
    print('p9.shape=',p9.shape)#p9=5
    print(p9)
    
    p10 = np.argmax(testLabel[0:5],axis=1)
    print(p10)

j=0#统计正确率
for i in range(0,5):
    if p10[i] == p9[i]:
         j+=1
print('ac=',j*100/5,'%')


Extracting C:\Users\TH\Desktop\DL\sample of writing numbers\train-images-idx3-ubyte.gz
Extracting C:\Users\TH\Desktop\DL\sample of writing numbers\train-labels-idx1-ubyte.gz
Extracting C:\Users\TH\Desktop\DL\sample of writing numbers\t10k-images-idx3-ubyte.gz
Extracting C:\Users\TH\Desktop\DL\sample of writing numbers\t10k-labels-idx1-ubyte.gz
trainData.shape= (5000, 784)
trainLabel.shape= (5000, 10)
testData.shape= (5, 784)
testLabel.shape= (5, 10)
[[0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]]
p1.shape= (5, 1, 784)
p2.shape= (5, 5000, 784)
p3.shape= (5, 5000)
p4.shape= (5, 5000)
p5.shape= (5, 4)
p6.shape= (5, 4)
[[-56.50591  -58.756847 -62.886257 -65.682335]
 [-55.839207 -56.380375 -59.76471  -62.925488]
 [-56.59217  -57.74902  -58.09804  -59.392155]
 [-52.729393 -54.639202 -56.909798 -57.933315]
 [-54.890205 -65.250984 -68.98432  -69.05096 ]]
[[3068   14 2531 3368]
